<h1 style="font-size:3rem;color:#325B74;">Lab 5</h1>

## Ethical aspects of data

<h2> The COMPAS dataset. What happens if we change the choice of variables? Can we make our decision more fair?</h2> 

Let's take our own decisions:

In [89]:
# filter dplyr warnings
%load_ext rpy2.ipython
import warnings
warnings.filterwarnings('ignore')

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [90]:
%%R
library(dplyr)
#You can choose your favorite option:
#a)Download the dataset and access it locally
raw_data <- read.csv("./compas-scores-two-years.csv") 
#b)Access the dataset directly from the repository
#raw_data <- read.csv("https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv")
nrow(raw_data)
colnames(raw_data)

 [1] "id"                      "name"                   
 [3] "first"                   "last"                   
 [5] "compas_screening_date"   "sex"                    
 [7] "dob"                     "age"                    
 [9] "age_cat"                 "race"                   
[11] "juv_fel_count"           "decile_score"           
[13] "juv_misd_count"          "juv_other_count"        
[15] "priors_count"            "days_b_screening_arrest"
[17] "c_jail_in"               "c_jail_out"             
[19] "c_case_number"           "c_offense_date"         
[21] "c_arrest_date"           "c_days_from_compas"     
[23] "c_charge_degree"         "c_charge_desc"          
[25] "is_recid"                "r_case_number"          
[27] "r_charge_degree"         "r_days_from_arrest"     
[29] "r_offense_date"          "r_charge_desc"          
[31] "r_jail_in"               "r_jail_out"             
[33] "violent_recid"           "is_violent_recid"       
[35] "vr_case_number"          

Note: if you obtain the following error: "UsageError: Cell magic `%%R` not found."
Try this solution: pip install rpy2


## Filtering of data

<em>In a 2009 study examining the predictive power of its COMPAS score, Northpointe defined recidivism as “a finger-printable arrest involving a charge and a filing for any uniform crime reporting (UCR) code.” We interpreted that to mean a criminal offense that resulted in a jail booking and took place after the crime for which the person was COMPAS scored.

It was not always clear, however, which criminal case was associated with an individual’s COMPAS score. To match COMPAS scores with accompanying cases, we considered cases with arrest dates or charge dates within 30 days of a COMPAS assessment being conducted. In some instances, we could not find any corresponding charges to COMPAS scores. We removed those cases from our analysis.

Next, we sought to determine if a person had been charged with a new crime subsequent to crime for which they were COMPAS screened. We did not count traffic tickets and some municipal ordinance violations as recidivism. We did not count as recidivists people who were arrested for failing to appear at their court hearings, or people who were later charged with a crime that occurred prior to their COMPAS screening.</em>

We do the same filtering that in the Propublica Study **BUT** we select different variables.

Finally we save the filtered csv file.



In [91]:
%%R
df <- dplyr::select(raw_data, age, c_charge_degree, race, age_cat, score_text, sex, priors_count, 
                    days_b_screening_arrest, decile_score, is_recid, two_year_recid, c_jail_in, c_jail_out,
                    juv_fel_count,juv_misd_count,juv_other_count,is_violent_recid) %>% 
        filter(days_b_screening_arrest <= 30) %>%
        filter(days_b_screening_arrest >= -30) %>%
        filter(is_recid != -1) %>%
        filter(is_violent_recid != -1) %>%
        filter(c_charge_degree != "O") %>%
        filter(score_text != 'N/A')
write.csv(df,"propublica_ext.csv")

nrow(df)

[1] 6172


Now we import the same libraries as in the previous labs.

In [92]:
#we import all the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # for plotting stuff
from random import seed, shuffle
from scipy.stats import multivariate_normal # for generating synthetic data 
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import confusion_matrix
SEED = 1122334455
seed(SEED) # set the random seed so that the random permutations can be reproduced again
np.random.seed(SEED)

We first read the filtered data. 

In [93]:
df = pd.read_csv("propublica_ext.csv")

print("Size of the dataset: %d" % len(df.index)) #We calculate the number of objects in the dataset




Size of the dataset: 6172


In [94]:
for col in df:
    print(df[col].unique())

[   1    2    3 ... 6170 6171 6172]
[69 34 24 44 41 43 39 27 23 37 47 31 25 64 21 32 26 33 30 55 49 29 51 35
 28 53 38 22 62 56 45 40 50 20 36 54 19 42 52 59 61 63 48 46 58 78 57 66
 70 60 65 68 71 83 67 75 72 74 96 73 80 77 76 79 18]
['F' 'M']
['Other' 'African-American' 'Caucasian' 'Hispanic' 'Asian'
 'Native American']
['Greater than 45' '25 - 45' 'Less than 25']
['Low' 'Medium' 'High']
['Male' 'Female']
[ 0  4 14  3  1  7  6  5 13  8  9 21  2 15 10 28 19 11 23 25 36 12 20 33
 16 18 17 22 30 24 27 26 37 29 31 38]
[ -1   0 -20  22  -2 -24 -13 -15 -10 -30  -4 -16 -26  -7  29  -3  23 -11
 -22 -21 -12  -8  -5 -23 -14  -9  -6 -27 -19 -18  26 -29  28   7 -17 -28
   9 -25  13  20  17  30   6   2  16   8   1  18  15  21  27  11   3  14
   5  24]
[ 1  3  4  6 10  5  9  2  7  8]
[0 1]
[0 1]
['2013-08-13 06:03:42' '2013-01-26 03:45:27' '2013-04-13 04:58:34' ...
 '2014-01-13 05:48:01' '2014-03-08 08:06:02' '2014-06-28 12:16:41']
['2013-08-14 05:41:20' '2013-02-05 05:36:53' '2013-04-14 07:02:04'

We need to binarize all the categorical features we considered in the dataset:

In [95]:
df = (
    pd.read_csv("propublica_ext.csv")
    #We first binarize the categorical feature c_charge_degree
    .assign(c_charge=lambda x:x['c_charge_degree'].replace({'F': 1, 'M':0}))
    #race
    .assign(african_american=lambda x:x['race'].replace({'Other': 0, 'African-American': 1, 'Caucasian': 0, 'Hispanic': 0, 'Asian': 0,'Native American': 0}))
    .assign(caucasian=lambda x:x['race'].replace({'Other': 0, 'African-American': 0, 'Caucasian': 1, 'Hispanic': 0, 'Asian': 0,'Native American': 0}))
    .assign(native_american=lambda x:x['race'].replace({'Other': 0, 'African-American': 0, 'Caucasian': 0, 'Hispanic': 0, 'Asian': 0,'Native American': 1}))
    .assign(hispanic=lambda x:x['race'].replace({'Other': 0, 'African-American': 0, 'Caucasian': 0, 'Hispanic': 1, 'Asian': 0,'Native American': 0}))
    .assign(asian=lambda x:x['race'].replace({'Other': 0, 'African-American': 0, 'Caucasian': 0, 'Hispanic': 0, 'Asian': 1,'Native American': 0}))
    .assign(other=lambda x:x['race'].replace({'Other': 1, 'African-American': 0, 'Caucasian': 0, 'Hispanic': 0, 'Asian': 0,'Native American': 0}))
    #age_cat
    .assign(less_than_25=lambda x:x['age_cat'].replace({'Greater than 45':0, '25 - 45':0, 'Less than 25':1}))
    .assign(between_25_45=lambda x:x['age_cat'].replace({'Greater than 45':0, '25 - 45':1, 'Less than 25':0}))
    .assign(greater_than_25=lambda x:x['age_cat'].replace({'Greater than 45':1, '25 - 45':0, 'Less than 25':0}))
    #score_text
    .assign(score_low=lambda x:x['score_text'].replace({'Low':1, 'Medium':0, 'High':0}))
    .assign(score_medium=lambda x:x['score_text'].replace({'Low':0, 'Medium':1, 'High':0}))
    .assign(score_high=lambda x:x['score_text'].replace({'Low':0, 'Medium':0, 'High':1}))
    #sex
    .assign(Male=lambda x:x['sex'].replace({'Male': 1, 'Female':0}))
    
)
DeleteList=['c_charge_degree','race','age_cat','score_text','sex','c_jail_in','c_jail_out']
df=df.drop(DeleteList, axis=1)
print("Dataset with %d" % df.shape[0],"objects and %d" % df.shape[1], "variables") #We calculate the number of objects in the dataset

df.head()


Dataset with 6172 objects and 25 variables


,Unnamed: 0,age,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count,...,hispanic,asian,other,less_than_25,between_25_45,greater_than_25,score_low,score_medium,score_high,Male
0,1,69,0,-1,1,0,0,0,0,0,...,0,0,1,0,0,1,1,0,0,1
1,2,34,0,-1,3,1,1,0,0,0,...,0,0,0,0,1,0,1,0,0,1
2,3,24,4,-1,4,1,1,0,0,1,...,0,0,0,1,0,0,1,0,0,1
3,4,44,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,1,0,0,1
4,5,41,14,-1,6,1,1,0,0,0,...,0,0,0,0,1,0,0,1,0,1


Actually, our dataset is only composed of 23 variables, since we do not include the first column as variable, and the variable "two_year_recid" is the binary label to predict.

We then define our data and our label:


In [96]:
display(df.columns)

Index(['Unnamed: 0', 'age', 'priors_count', 'days_b_screening_arrest',
       'decile_score', 'is_recid', 'two_year_recid', 'juv_fel_count',
       'juv_misd_count', 'juv_other_count', 'is_violent_recid', 'c_charge',
       'african_american', 'caucasian', 'native_american', 'hispanic', 'asian',
       'other', 'less_than_25', 'between_25_45', 'greater_than_25',
       'score_low', 'score_medium', 'score_high', 'Male'],
      dtype='object')

We now select from this list only the variables we want to consider in our classification problem and the corresponding labels:

In [97]:
# Here is a way to select these columns using the column names
    
#feature_columns = ['Number_of_Priors', 'score_factor','Age_Above_FourtyFive', 'Age_Below_TwentyFive', 'African_American','Asian', 'Hispanic', 'Native_American', 'Other', 'Female',       'Misdemeanor']

feature_columns = ['age', 'priors_count', 'days_b_screening_arrest',
         'juv_fel_count',
       'juv_misd_count', 'juv_other_count', 'is_violent_recid', 'c_charge',
        'less_than_25', 'between_25_45', 'greater_than_25',
        'Male']


data = df[feature_columns].values
y = df['two_year_recid'].values
df=df.assign(COMPAS_Decision=lambda x:x['score_low'].replace({0: 1, 1:0}))
y_compas = df['COMPAS_Decision'].values

In [98]:
#We load the libraries for the SVM

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# shuffle the data
#n_samples=data.shape[0]
#perm = list(range(0,n_samples))
#shuffle(perm)
#data = data[perm]
#y = y[perm]
#y_compas=y_compas[perm]


#Create train and validation set
#train_x, test_x, train_y, test_y = train_test_split(data, y, test_size=0.1, shuffle=True, stratify=y, random_state=42)

svm_clf = SVC(kernel="linear", C=1.0)
svm_clf.fit(data, y)

y_pred=svm_clf.predict(data)


#svm_clf = SVC(kernel="linear", C=1.0)
#svm_clf.fit(train_x, train_y)

#ytrain_pred=svm_clf.predict(train_x)
#ytest_pred=svm_clf.predict(test_x)



In [99]:
b_recid = df[df['african_american'] == 1]
w_recid = df[df['caucasian'] == 1]
print(
    ' Accuracy SVM (All):  \t %.2f' % (metrics.accuracy_score(y, y_pred)*100), "%\n",
    'Accuracy SVM (Black):\t %.2f' % (metrics.accuracy_score(y_pred[df['african_american'] == 1], b_recid['two_year_recid'])*100),"%\n", 
    'Accuracy SVM (White):\t %.2f' % (metrics.accuracy_score(y_pred[df['caucasian'] == 1], w_recid['two_year_recid'])*100),"%\n",

)

pd.crosstab(y_pred, df['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')

FPR_s=pd.crosstab(y_pred, df['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[0][1]
FNR_s=pd.crosstab(y_pred, df['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[1][0].astype('float')

print("FPR SVM %.2f" % (FPR_s*100),"%")
print("FNR SVM %.2f" % (FNR_s*100),"%")




 Accuracy SVM (All):  	 70.01 %
 Accuracy SVM (Black):	 68.57 %
 Accuracy SVM (White):	 71.18 %

FPR SVM 9.60 %
FNR SVM 54.40 %


## Questions:

**1-** Provide the accuracy for the COMPAS algorithm.

**2-** Provide the accuracy for black defendants for the COMPAS algorithm.

**3-** Provide the accuracy for white defendants for the COMPAS algorithm.

**4-** Provide the FPR for the COMPAS algorithm.

**5-** Provide the FNR for the COMPAS algorithm.



In [100]:
#1- Provide the accuracy for the COMPAS algorithm.
#Accuracy


print("Accuracy: %.1f" % (metrics.accuracy_score(y, y_compas)*100), "%") #you need to verify

Accuracy: 66.1 %


In [101]:
#64.91
print("Accuracy: %.1f" % (metrics.accuracy_score(y[df['african_american'] == 1], y_compas[df['african_american'] == 1])*100), "%") #you need to verify

Accuracy: 64.9 %


In [102]:
#67.19
#3- Provide the accuracy for white defendants for the COMPAS algorithm.
print("Accuracy: %.1f" % (metrics.accuracy_score(y[df['caucasian'] == 1], y_compas[df['caucasian'] == 1])*100), "%") #you need to verify

Accuracy: 67.2 %


In [103]:
#4
#30.27
#Provide the FPR for the COMPAS algorithm.
FPR_s=pd.crosstab(y_compas, df['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[0][1]
print("FPR Compass %.2f" % (FPR_s*100),"%")

FPR Compass 30.27 %


In [104]:
#5- Provide the FNR for the COMPAS algorithm.
#38.31
FNR_s=pd.crosstab(y_compas, df['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[1][0].astype('float')
print("FNR Compass %.2f" % (FNR_s*100),"%")

FNR Compass 38.31 %


In [105]:
#We calculate FPR and FNR for black defendants for the COMPAS algorithm:
FPR_b=pd.crosstab(b_recid['COMPAS_Decision'], b_recid['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[0][1]
FNR_b=pd.crosstab(b_recid['COMPAS_Decision'], b_recid['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[1][0].astype('float')

print("FPR of Black %.2f" % (FPR_b*100),"%")
print("FNR of Black %.2f" % (FNR_b*100),"%")



FPR of Black 42.34 %
FNR of Black 28.48 %


In [106]:
#exercise 6 and 7
FPR_b=pd.crosstab(w_recid['COMPAS_Decision'], w_recid['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[0][1]
FNR_b=pd.crosstab(w_recid['COMPAS_Decision'], w_recid['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[1][0].astype('float')

print("FPR of White %.2f" % (FPR_b*100),"%")
print("FNR of White %.2f" % (FNR_b*100),"%")

FPR of White 22.01 %
FNR of White 49.64 %


In [107]:
#Exercise 8 and 9

#"FPR of Black is 1.9 times greater than for White"
print(42.34/22.01)
#"FNR of Black is 1.7 times smaller than for White"
print(49.64/28.48)


1.9236710586097228
1.7429775280898876


## Questions:

For the COMPAS algorithm:

**6-** Calculate the FPR for white defendants.

**7-** Calculate the FNR for white defendants.

**8-** Replace **x** by the right number in the two following statement:

"FPR of Black is **x** times greater than for White"

**9-** Replace **x** by the right number in the two following statement:

"FNR of Black is **x** times smaller than for White"



In [108]:
#For the SVM:
FPR_b=pd.crosstab(y_pred[df['african_american'] == 1], b_recid['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[0][1]
FNR_b=pd.crosstab(y_pred[df['african_american'] == 1], b_recid['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[1][0].astype('float')

print("FPR of Black %.2f" % (FPR_b*100),"%")
print("FNR of Black %.2f" % (FNR_b*100),"%")

FPR of Black 13.74 %
FNR of Black 47.56 %


In [109]:
#10- Calculate the FPR for white defendants.
#11- Calculate the FNR for white defendants.
#For the SVM:
FPR_b=pd.crosstab(y_pred[df['caucasian'] == 1], w_recid['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[0][1]
FNR_b=pd.crosstab(y_pred[df['caucasian'] == 1], w_recid['two_year_recid'], rownames=['Predicted recividism'],colnames=['Actual recividism'],normalize='columns')[1][0].astype('float')

print("FPR of White %.2f" % (FPR_b*100),"%")
print("FNR of White %.2f" % (FNR_b*100),"%")

FPR of White 6.48 %
FNR of White 63.63 %


In [110]:
#12- Replace x by the right number in the two following statement:

#"FPR of Black is x times greater than for White"
13.74/6.48

2.1203703703703702

In [111]:
#13- Replace x by the right number in the two following statement:

#"FNR of Black is x times smaller than for White"
47.56/63.63

0.7474461731887474

In [112]:
#15- Which is the best solution in terms of accuracy? Is it fair (in terms of accuracy)?
#the SVM is biger and so it's better

In [113]:
#16- Which is the best solution in terms of FPR? Based on answers 8 and 12,
#which solution is more fair (in terms of FPR)?
#for FPR,SVM is smaller, so it's more fair

In [ ]:
#17- Which is the best solution in terms of FNR? Based on answers 9 and 13, 
#which solution is more fair (in terms of FNR)?
#FNR in Compass is smaller, so it's more fair

In [123]:
def equal_opportunity(y,y_pred, x_prot):
    Pos_pro=0.0
    Pos_nonpro=0.0
    PPos_pro=0.0
    PPos_nonpro=0.0
    
    n=y_pred.size

    for i in range(0,n):
        if (y[i]==1 and x_prot[i]==0):
            Pos_nonpro=Pos_nonpro+1
            if (y_pred[i]==1):
                PPos_nonpro=PPos_nonpro+1
        if (y[i]==1 and x_prot[i]==1):
            Pos_pro=Pos_pro+1
            if (y_pred[i]==1):
                PPos_pro=PPos_pro+1
    #print(Pos_nonpro)
    #print(Pos_pro)
    #print(PPos_pro)
    #print(PPos_nonpro)
    UNF_EOpp=abs((PPos_nonpro/Pos_nonpro)-(PPos_pro/Pos_pro))
    
    return UNF_EOpp

def predictiv_equality(y,y_pred, x_prot):
    Pos_pro=0.0
    Pos_nonpro=0.0
    PPos_pro=0.0
    PPos_nonpro=0.0
    
    n=y_pred.size

    for i in range(0,n):
        if (y[i]==0 and x_prot[i]==0):
            Pos_nonpro=Pos_nonpro+1
            if (y_pred[i]==1):
                PPos_nonpro=PPos_nonpro+1
        if (y[i]==0 and x_prot[i]==1):
            Pos_pro=Pos_pro+1
            if (y_pred[i]==1):
                PPos_pro=PPos_pro+1
    #print(Pos_nonpro)
    #print(Pos_pro)
    #print(PPos_pro)
    #print(PPos_nonpro)
    UNF_PE =abs((PPos_nonpro/Pos_nonpro)-(PPos_pro/Pos_pro))
    
    return UNF_PE

def equalized_odds(y,y_pred, x_prot):
    return equal_opportunity(y,y_pred, x_prot) + predictive_equality(y,y_pred, x_prot)

def predictiv_parity(y,y_pred, x_prot):
    Pos_pro=0.0
    Pos_nonpro=0.0
    PPos_pro=0.0
    PPos_nonpro=0.0
    
    n=y_pred.size

    for i in range(0,n):
        if (y_pred[i]==1 and x_prot[i]==0):
            Pos_nonpro=Pos_nonpro+1
            if (y[i]==1):
                PPos_nonpro=PPos_nonpro+1
        if (y_pred[i]==1 and x_prot[i]==1):
            Pos_pro=Pos_pro+1
            if (y[i]==1):
                PPos_pro=PPos_pro+1
    #print(Pos_nonpro)
    #print(Pos_pro)
    #print(PPos_pro)
    #print(PPos_nonpro)
    UNF_PE =abs((PPos_nonpro/Pos_nonpro)-(PPos_pro/Pos_pro))
    
    return UNF_PE
def statistical_parity(y,y_pred, x_prot):
    Pos_pro=0.0
    Pos_nonpro=0.0
    PPos_pro=0.0
    PPos_nonpro=0.0
    
    n=y_pred.size

    for i in range(0,n):
        if (x_prot[i]==0):
            Pos_nonpro=Pos_nonpro+1
            if (y_pred[i]==1):
                PPos_nonpro=PPos_nonpro+1
        if (x_prot[i]==1):
            Pos_pro=Pos_pro+1
            if (y_pred[i]==1):
                PPos_pro=PPos_pro+1
    #print(Pos_nonpro)
    #print(Pos_pro)
    #print(PPos_pro)
    #print(PPos_nonpro)
    UNF_SP=abs((PPos_nonpro/Pos_nonpro)-(PPos_pro/Pos_pro))
    
    return UNF_SP

def disparate_impact(y,y_pred, x_prot):
    Pos_pro=0.0
    Pos_nonpro=0.0
    PPos_pro=0.0
    PPos_nonpro=0.0
    
    n=y_pred.size

    for i in range(0,n):
        if (x_prot[i]==0):
            Pos_nonpro=Pos_nonpro+1
            if (y_pred[i]==1):
                PPos_nonpro=PPos_nonpro+1
        if (x_prot[i]==1):
            Pos_pro=Pos_pro+1
            if (y_pred[i]==1):
                PPos_pro=PPos_pro+1
    #print(Pos_nonpro)
    #print(Pos_pro)
    #print(PPos_pro)
    #print(PPos_nonpro)
    UNF_DI=min((PPos_pro/Pos_pro)/(PPos_nonpro/Pos_nonpro), (PPos_nonpro/Pos_nonpro)/(PPos_pro/Pos_pro))
    
    return UNF_DI



In [125]:
#18- Calculate the 6 fairness metrics for the COMPAS classifier.
UNF_EOpp=equal_opportunity(y,y_compas,df['african_american'])
UNF_PE=predictiv_equality(y,y_compas,df['african_american'])
UNF_EOd=equalized_odds(y,y_compas,df['african_american'])
UNF_PP=predictiv_parity(y,y_compas,df['african_american'])
UNF_SP=statistical_parity(y,y_compas,df['african_american'])
UNF_DI=disparate_impact(y,y_compas,df['african_american'])
liste=[UNF_EOpp,UNF_PE,UNF_EOd,UNF_PP,UNF_SP,UNF_DI]
mat = ["EO", "PE", "EOD", "PP", "SP", "DI"]

pd.DataFrame({"metrics": mat, "values": liste})

,metrics,values
0,EO,0.240493
1,PE,0.219488
2,EOD,0.459981
3,PP,0.058429
4,SP,0.268422
5,DI,0.534041


In [126]:
#Calculate the 6 fairness metrics for the SVM classifier.
UNF_EOpp=equal_opportunity(y,y_pred,df['african_american'])
UNF_PE=predictiv_equality(y,y_pred,df['african_american'])
UNF_EOd=equalized_odds(y,y_pred,df['african_american'])
UNF_PP=predictiv_parity(y,y_pred,df['african_american'])
UNF_SP=statistical_parity(y,y_pred,df['african_american'])
UNF_DI=disparate_impact(y,y_pred,df['african_american'])
liste=[UNF_EOpp,UNF_PE,UNF_EOd,UNF_PP,UNF_SP,UNF_DI]
mat = ["EO", "PE", "EOD", "PP", "SP", "DI"]

pd.DataFrame({"metrics": mat, "values": liste})

,metrics,values
0,EO,0.167240
1,PE,0.075189
2,EOD,0.242429
3,PP,0.026277
4,SP,0.164667
5,DI,0.515460


## Questions:

For the SVM classifier:

**10-** Calculate the FPR for white defendants.


**11-** Calculate the FNR for white defendants.

**12-** Replace **x** by the right number in the two following statement:

"FPR of Black is **x** times greater than for White"

**13-** Replace **x** by the right number in the two following statement:

"FNR of Black is **x** times smaller than for White"

**14-** Fill in the following table:

$$
\begin{array}{cc}
& \text{SVM }&\text{COMPAS }\\
&\begin{array}{cccc}
\hline \hline 
&\text { All } & \text { Black } & \text { White }\\
\hline 
\text{Accuracy}& {70.01}&{68.57}  &{71.18}  \\
\text{FPR}&{9.60} &{13.74}  & {6.48} \\
\text{FNR}&{54.40} &{47.56}  & {63.63} \\
\hline
\end{array}
&\begin{array}{cccc}
\hline \hline 
&\text { All } & \text { Black } & \text { White }\\
\hline 
\text{Accuracy}&{66.1} &{64.9}  &{67.2}  \\
\text{FPR}&{30.27} &{42.34}  &{22.01}  \\
\text{FNR}&{38.31} &{28.48}  &{49.64}  \\
\hline
\end{array}\\
\end{array}
$$

FPR of White 22.01 %
FNR of White 49.64 %

**15-** Which is the best solution in terms of accuracy? Is it fair (in terms of accuracy)?

**16-** Which is the best solution in terms of FPR? Based on answers 8 and 12, which solution is more fair (in terms of FPR)?

**17-** Which is the best solution in terms of FNR? Based on answers 9 and 13, which solution is more fair (in terms of FNR)?

**18-** Calculate the 6 fairness metrics for the COMPAS classifier.

**19-** Calculate the 6 fairness metrics for the SVM classifier.

**20-** As a future (or actual) data scientist, which solution would you choose for **this** specific problem? Justify your answer.

In [ ]:
#I'll choose SVM, because If I see the accuracy, in SVM is higher accuracy
#In FPR is lower, thats mean its better 